- selenium 爬取淘宝商品信息的基本操作思路
    
    1 登陆
    
    2 拉动下滑条
    
    3 采集数据
    
    4 下一页
    
    5 再拉动下滑条
    
    6 再采集数据
    
    7 再翻页

In [ ]:
from selenium import webdriver
import time
import pandas as pd

# 找到输入框，找到搜索按钮
# 元素定位
# 定义一个商品搜索函数
def search_product():
    driver.find_element_by_xpath('//*[@id="q"]').send_keys(kw)
    driver.find_element_by_xpath('//*[@id="J_TSearchForm"]/div[1]/button').click()
    time.sleep(1)

    # 点击扫码登陆按钮
    driver.find_element_by_xpath('//*[@id="login"]/div[1]/i').click()

    # 沉睡 10s 用来为手机扫码登陆的预留时间
    time.sleep(10)

# 定义一个下拉滑动函数
def drop_down():
    # 一次拉一部分，每拉一次暂停一小会
    for x in range(1,11,2):
        time.sleep(0.5)      
        # j 代表滑动条位置
        j = x/10        
        # 执行 js 语法
        js = "document.documentElement.scrollTop = document.documentElement.scrollHeight * %f" % j
        driver.execute_script(js)

# 定义一个采集数据的函数
def get_product():
    titles = []
    divs = driver.find_elements_by_xpath('//div[@class = "items"]/div[@data-category = "auctions"]')
    for div in divs:
        title = div.find_element_by_xpath('.//div[@class = "row row-2 title"]/a').text
        price = div.find_element_by_xpath('.//div[@class = "price g_price g_price-highlight"]/strong').text + "元"
        deal = div.find_element_by_xpath('.//div[@class = "deal-cnt"]').text
        img = div.find_element_by_xpath('.//div[@class = "pic"]/a/img').get_attribute("src")
        shopname = div.find_element_by_xpath('.//div[@class = "shop"]/a/span[2]').text
        position = div.find_element_by_xpath('.//div[@class = "location"]').text
        
        titles.append(title)
        
        data = pd.DataFrame()
        data["标题"] = titles
        
        product = {
            "标题":title,
            "价格":price,
            "月销量":deal,
            "店铺名":shopname,
            "店铺地点":position,
            "图片链接":img,
        }
        print(product)
    
# 定义一个翻页函数
def next_page():
    get_product()
    drop_down()
    num = 1
    while num != 100:
        driver.get("https://s.taobao.com/search?q={}&s={}".format(kw,44*num))
        
        # 无限循环进入网页，可能会造成网页卡顿
        driver.implicitly_wait(10)
        # 智能等待，最高等待时间为 10s ，否则抛出异常
        
        num += 1
        get_product()
        drop_down()

    
if __name__ == "__main__":
    
    # 参数动态化
    kw = input("请输入你要搜索的商品：")
    
    #实例化一个浏览器对象(传入浏览器的驱动程序)
    driver_path = r"E:\chromedriver\chromedriver.exe"    #我将chrome驱动安装的路径
    driver = webdriver.Chrome(executable_path = driver_path)

    #让浏览器发起一个指定url对应的请求
    driver.get("https://www.taobao.com/")
    
    # 调用 : 商品搜索函数 和 下拉滑动函数 翻页函数
    search_product()
    next_page()
    
    
   

    

# 注意： 元素定位有个坑，如果元素没有加载出来，那么我们就不能定位到没有加载的数据
# 解决方案： 使用 js 做滑动条操作

请输入你要搜索的商品：华为手机
{'标题': '【8+256直降200元】华为HONOR 荣耀20手机正品全面屏超广角AI四摄', '价格': '2299.00元', '月销量': '104人付款', '店铺名': '天猫超市', '店铺地点': '上海', '图片链接': 'https://g-search1.alicdn.com/img/bao/uploaded/i4/i2/725677994/O1CN01Zeb3cT28vIkkwvhr5_!!725677994-0-sm.jpg_230x230.jpg_.webp'}
{'标题': '【下单减100】华为HONOR荣耀9X手机麒麟810芯片升降全面官方正品', '价格': '1599.00元', '月销量': '1963人付款', '店铺名': '天猫超市', '店铺地点': '上海', '图片链接': 'https://g-search1.alicdn.com/img/bao/uploaded/i4/i4/168/O1CN012RLCPy1D6z8vki45w_!!168-0-lubanu.jpg_230x230.jpg_.webp'}
{'标题': '咨询优惠【24期分期/送果汁机】HUAWEI P40 Pro 5G新款手机官方旗舰店mate30pro5g版正品p40pro直降p30', '价格': '4188.00元', '月销量': '177人付款', '店铺名': '博盛景数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g-search1.alicdn.com/img/bao/uploaded/i4/i4/1710936647/O1CN01bDoMwr1yyNFmkc7qi_!!0-item_pic.jpg_230x230.jpg_.webp'}
{'标题': '领100元券 24期免息顺丰当天发【新品Huawei/华为 nova 5i手机官网官方旗舰店正品非5G降价mate30pro', '价格': '1599.00元', '月销量': '1499人付款', '店铺名': '小淘器数码专营店', '店铺地点': '安徽 合肥', '图片链接': 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i3/168463

{'标题': '新品【24期分期免息】华为HONOR荣耀30S 5G全网通手机官方旗舰店官网正品20降价系列V30pro/X10/nova7se 256G', '价格': '2399.00元', '月销量': '460人付款', '店铺名': '同迅数码专营店', '店铺地点': '辽宁 大连', '图片链接': 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i3/1666576242/O1CN01PhE8pV1vysqRGWxIU_!!0-item_pic.jpg_230x230.jpg_.webp'}
{'标题': '限量100元券【6期免息+好礼】Huawei/华为nova7 SE手机5G版华为官方旗舰正品直降nova6se官网荣耀30s', '价格': '2399.00元', '月销量': '1690人付款', '店铺名': '创汇通达数码旗舰店', '店铺地点': '北京', '图片链接': 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i2/3063905773/O1CN01vTw8YE1sW55lvj9Al_!!0-item_pic.jpg_230x230.jpg_.webp'}
{'标题': '24期免息【当天发货】Huawei/华为 P40 Pro 5G手机官方旗舰店华为p40pro全网通官网正品mate30新品nova7上市', '价格': '4188.00元', '月销量': '100人付款', '店铺名': '京联通达数码旗舰店', '店铺地点': '北京', '图片链接': 'https://g-search1.alicdn.com/img/bao/uploaded/i4/i3/2924891464/O1CN01fxcsbF1MgYRdmhid4_!!0-item_pic.jpg_230x230.jpg_.webp'}
{'标题': '华为老年手机智能荣耀大屏大字大声音老人机电信超长待机全网通4g', '价格': '699.00元', '月销量': '1579人付款', '店铺名': '王玉玉1234', '店铺地点': '上海', '图片链接': 'https://g-search3.alicdn.com/img/bao/uploa

{'标题': '5G新品24期免息当天发Huawei/华为 nova7 Pro 5G官方旗舰正品手机8全网通新款p40荣耀nova7pro官网免息nova6', '价格': '3699.00元', '月销量': '1204人付款', '店铺名': '广东电信亿品汇专卖店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '24期免息当天发直降300元 Huawei/华为 P40 pro手机5G全网通官方旗舰正品直降p30新款mate30pro荣耀p40pro', '价格': '4188.00元', '月销量': '829人付款', '店铺名': '广东电信亿品汇专卖店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '优惠350【24期免息/当天发】华为/HUAWEI P40 5G版华为官方旗舰5g新款p40正品直降官网p40pro 5g荣耀', '价格': '4188.00元', '月销量': '434人付款', '店铺名': '创汇通达数码旗舰店', '店铺地点': '北京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '当天发货【6期免息】Huawei/华为 nova 7 5G手机官方旗舰店nova7手机官网正品新款p40pro直降荣耀mate30', '价格': '2399.00元', '月销量': '1094人付款', '店铺名': '京联通达数码旗舰店', '店铺地点': '北京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '华为nova7pro Huawei/华为 nova 7 Pro 5G手机官方nova7se 7i 6se', '价格': '2068.00元', '月销量': '668人付款', '店铺名': 'wezebaba', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【到手价2958元】Huawei 华为 P30 手机官方旗舰店 畅享 nova 降价 mate30 P40 Pro 顺丰速发', '价格': 

{'标题': '【6+128GB优惠100+24期免息】华为旗下荣耀30新品5G手机50倍超稳远摄麒麟985芯片李现同款智能官方旗舰', '价格': '2999.00元', '月销量': '1.5万+人付款', '店铺名': '荣耀官方旗舰店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【爆款直降】HUAWEI/华为Mate30 8+128GB 全网通4G智能手机麒麟990徕卡拍照旗舰官方正品', '价格': '3299.00元', '月销量': '1785人付款', '店铺名': '苏宁易购官方旗舰店', '店铺地点': '江苏 南京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【百亿补贴专享价】HUAWEI/华为Mate30 Pro 8+128GB 麒麟990芯曲面屏双4000万徕卡四摄4G智能mate30pro', '价格': '4599.00元', '月销量': '2188人付款', '店铺名': '苏宁易购官方旗舰店', '店铺地点': '江苏 南京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【限时优惠低至2499元】华为旗下荣耀V30双模5G麒麟990旗舰芯片李现同款PRO拍照官方旗舰店', '价格': '2999.00元', '月销量': '781人付款', '店铺名': '荣耀官方旗舰店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '咨询优惠【当天发+6期免息】Huawei/华为 Nova 7 5G手机官方nova6se官网5荣耀30s直降mate30正品p40pro旗舰店', '价格': '2999.00元', '月销量': '1653人付款', '店铺名': '深蓝港数码旗舰店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '新品【当天发+送好礼 】华为Honor/荣耀Play4T手机新品6+128GB大内存AI/摄影官方正品旗舰店30全新SX游戏', '价格': '1099.00元', '月销

{'标题': '【直降450】 honor/荣耀 荣耀30S 5G手机v30华为荣耀30pro5g官方S', '价格': '1928.00元', '月销量': '383人付款', '店铺名': '友铭数码通讯', '店铺地点': '上海', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【领370元券至高减400】华为HONOR 荣耀20青春版手机官方旗舰店20i正品8x降价9xpro荣耀10青春版20s新款V30', '价格': '959.00元', '月销量': '171人付款', '店铺名': '能良数码官方旗舰店', '店铺地点': '上海', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '顺丰速发【花呗分期】Huawei/华为nova5i全网通华为官方旗舰店正品华为学生机nova4e官网直降荣耀9x', '价格': '1268.00元', '月销量': '464人付款', '店铺名': '创汇通达数码旗舰店', '店铺地点': '北京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '领300元券可减320元】华为HONOR 荣耀20青春版手机官方旗舰店20i正品8x降价9xpro荣耀10青春版20s新款热卖V30', '价格': '999.00元', '月销量': '1385人付款', '店铺名': '三际数码官方旗舰店', '店铺地点': '山东 济南', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '减100元【咨询更优惠 6期免息 送豪礼】Huawei华为nova 7 SE|5G手机nova7pro官方旗舰店SE官网P40正品mate30', '价格': '2399.00元', '月销量': '318人付款', '店铺名': '华为莫问专卖店', '店铺地点': '浙江 杭州', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '直降600元分期免息现货发顺丰Huawei华为nova5ipro全网通4G华为官方旗舰店nova6se/p30pro/mate30pro', '价格': '1329.00元', '月销量': '294

...

{'标题': '华为mate30pro手机壳真皮翻盖式mate30智能视窗保护套4g/5g限量版魅特m30皮套全包防摔曲屏紫色女高档橙外套p', '价格': '186.00元', '月销量': '1160人付款', '店铺名': '欧德卡数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': 'Huawei/华为 华为MatePad平板电脑10.4英寸pro新款m6安卓全网通话', '价格': '1798.00元', '月销量': '261人付款', '店铺名': '上海数码馆', '店铺地点': '上海', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【到手价799】华为荣耀手表2 Magic Watch3智能运动可通话魔法系列男移动电子多功能女防水正品适用小米gt2', '价格': '999.00元', '月销量': '1936人付款', '店铺名': '得者淘数码专营店', '店铺地点': '浙江 杭州', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '罗马仕20000毫安充电宝大容量双向快充手机罗马 仕旗舰店官方正品适用小米苹果移动电源22.5/18W闪充专用', '价格': '99.00元', '月销量': '1.5万+人付款', '店铺名': '北京七千猫数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '马卡龙粉色真无线蓝牙耳机双耳入耳式女生款可爱少女隐形迷你适用于苹果vivo小米安卓通用型原装耳塞oppo', '价格': '68.00元', '月销量': '1.0万+人付款', '店铺名': '丰乔数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '手机稳定器通用型一体式自拍杆适用华为苹果拍照神器蓝牙自排三脚架vlog拍摄直播支架手持防抖平衡云台稳定器', '价格': '209.00元', '月销量': '1246人付款', '店铺名': '硕图旗舰店', '店铺地点': '广东 深圳', '图片链

{'标题': '24小时速发 Huawei/华为 nova 2s全面屏4G全网通华为 NOVA4E 华为官方旗舰店正品畅享8plus麦芒6', '价格': '1099.00元', '月销量': '16人付款', '店铺名': '旭翼数码专营店', '店铺地点': '上海', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【咨询优惠】HUAWEI 华为畅享10 plus华为官方旗舰授权正品4G手机', '价格': '1219.00元', '月销量': '8人付款', '店铺名': '恒发通讯88888', '店铺地点': '山东 济南', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': 'Huawei/华为 畅享7 Plus 高配 老人智能机大字大屏全网通4G老人机', '价格': '300.00元', '月销量': '35人付款', '店铺名': 'junnian20', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '减2000元 24期免息 华为 HUAWEI Mate Xs 可折叠全面屏 5G双模 海思 麒麟 990处理器 动联通电信5G全网通', '价格': '21999.00元', '月销量': '2人付款', '店铺名': '大唐回浦数码旗舰店', '店铺地点': '浙江 台州', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': 'Huawei/华为 畅享 10e长续航5000大电池128G大内存珍珠屏智能手机', '价格': '825.00元', '月销量': '6人付款', '店铺名': '成都一凡恒创科技有限公司', '店铺地点': '', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '新品华为/HUAWEI nova 5z快充AI四摄4800万麒麟810智能机nova5z', '价格': '1258.00元', '月销量': '5人付款', '店铺名': '东胜科技商城', '店铺地点': '四川 成都', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': 

{'标题': '全新旗舰Huawei/华为 p40 pro 5G华为p40麒麟990官方正品假一罚十', '价格': '3688.00元', '月销量': '3人付款', '店铺名': 'tb118754094', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【百亿补贴/速发】Huawei/华为 nova 7 5G手机新品官方旗舰店nova6pro荣耀30直降p40全mate305G版正品se', '价格': '2999.00元', '月销量': '6人付款', '店铺名': '玩风数码专营店', '店铺地点': '山东 济南', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '当天发货【5G至高咨询减800元】Huawei/华为Mate30 Pro手机Mate30官方旗舰店nova7 P30官网mate20折叠XS P40', '价格': '4599.00元', '月销量': '11人付款', '店铺名': '华为莫问专卖店', '店铺地点': '浙江 杭州', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '官网降1000】p40手机Huawei/华为 p40 pro 5G麒麟990四摄50倍变焦', '价格': '3688.00元', '月销量': '6人付款', '店铺名': 'ldtxsb', '店铺地点': '上海', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '华为畅享10S现货新品HUAWEI 畅享 10S全新正品全网通4G手机8+128', '价格': '1188.00元', '月销量': '7人付款', '店铺名': '度伊特2009', '店铺地点': '安徽 合肥', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【限时3期免息+200元优惠券】Huawei/华为 nova 5 Pro全网通智能官方授权店正品nova5pro直降mate30新款', '价格': '2499.00元', '月销量': '4人付款', '店铺名': '武汉电信旗舰', '店铺地点': '湖北 武汉', '

{'标题': '【新品现货】HUAWEI 华为畅享10 plus手机官方旗舰10s正品10e', '价格': '1228.00元', '月销量': '9人付款', '店铺名': '中宽通讯行', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '分期免息Huawei/华为 P10全网通4G手机 双卡双待华为P10plus手机', '价格': '585.00元', '月销量': '40人付款', '店铺名': '麒睿通讯', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【新品 现货速发】华为HONOR 荣耀30 Pro/荣耀30 Pro+新品5G手机50倍远摄拍照官方V30 9x 同城闪送', '价格': '3999.00元', '月销量': '1人付款', '店铺名': '正飞中博数码旗舰店', '店铺地点': '福建 厦门', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': 'Huawei/华为 麦芒6全面屏手机官方正品全网通4G 顺丰包邮送豪礼', '价格': '650.00元', '月销量': '32人付款', '店铺名': '国冠数码商诚01', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '送手环分期免息】Huawei/华为畅享10plus手机畅享10 plus的想销cx荣耀10plus官方旗舰店麦芒8nova5z 10e', '价格': '1399.00元', '月销量': '4人付款', '店铺名': '年年盛数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '当天发优惠50元24期分期Huawei/华为畅享Z 5G官方旗舰店5g手机正品畅想z 10 plus新款nova7荣耀p40直降9畅享z', '价格': '1899.00元', '月销量': '5人付款', '店铺名': '同步飞扬数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.

{'标题': '[5G新品/分期免息/同城闪送]HUAWEI 华为 P40 5G麒麟990 5G超感知徕卡三摄30倍数字变焦正品智能', '价格': '4488.00元', '月销量': '3人付款', '店铺名': '王者天创旗舰店', '店铺地点': '海南 海口', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '华为nova5天津HUAWEI nova 5i nova5ipro 全网通4G智能手机nova5p', '价格': '1248.00元', '月销量': '22人付款', '店铺名': 'baby家衣', '店铺地点': '天津', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '华为手机honor/荣耀 荣耀V30 Pro荣耀V30pro5G版全网通新款', '价格': '2498.00元', '月销量': '4人付款', '店铺名': 'chenzhenshan520', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '新款华为 HUAWEI nova 6/nova 6 5G 麒麟990 5G/4G 手机正品', '价格': '1523.00元', '月销量': '5人付款', '店铺名': '风火电讯', '店铺地点': '山东 济南', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '【现货速发】Huawei/华为 nova 5 Pro手机5g nova6直降6se新款mate30正品12荣耀40', '价格': '1999.00元', '月销量': '11人付款', '店铺名': '民讯通数码专营店', '店铺地点': '北京', '图片链接': 'https://g.alicdn.com/s.gif'}
{'标题': '活动直降/Huawei/华为 畅享10 手机 华为官方旗舰店正品畅想10p 畅享10e 学生老年人智能千元机', '价格': '999.00元', '月销量': '110人付款', '店铺名': '博海宇通数码专营店', '店铺地点': '广东 深圳', '图片链接': 'https://g.alicdn.com/s

- 注意： 元素定位有个坑，如果元素没有加载出来，那么我们就不能定位到没有加载的数据
- 解决方案： 使用 js 代码做滑动条操作